In [1]:
from copy import deepcopy

import torch
import torch.nn as nn
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import math
import os, sys
from tqdm import tqdm
import random

from denoising_diffusion_pytorch import GaussianDiffusion
from denoising_diffusion_pytorch.denoising_diffusion_pytorch import SinusoidalPosEmb, unnormalize_to_zero_to_one
from denoising_diffusion_pytorch.progressive_distillation import ProgressiveDistillationGaussianDiffusion


C:\Users\lukov\miniconda3\envs\dd\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

class OneDModel(torch.nn.Module):
    def __init__(self, dim=16, **kw):
        super().__init__(**kw)
        self.channels = self.out_dim = 1
        self.self_condition = False
        time_dim = dim * 4

        sinu_pos_emb = SinusoidalPosEmb(dim)

        self.time_mlp = nn.Sequential(
            sinu_pos_emb,
            nn.Linear(dim, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, dim)
        )

        self.main = torch.nn.ModuleList([
            torch.nn.Linear(1, dim),
            torch.nn.GELU(),
            torch.nn.Linear(dim, dim),
            torch.nn.GELU(),
            torch.nn.Linear(dim, dim),
            torch.nn.GELU(),
            torch.nn.Linear(dim, dim),
            torch.nn.GELU(),
            torch.nn.Linear(dim, 1)
        ])

    def forward(self, x_t, t, self_cond=None):
        """ x: (batsize, 1, 1, 1)"""
        x_t = x_t.squeeze(-1).squeeze(-1)

        temb = self.time_mlp(t)

        h = self.main[0](x_t)
        h = h + temb
        for layer in self.main[1:]:
            h = layer(h)

        h = h[:, :, None, None]
        return h


class OneDDataset(torch.utils.data.Dataset):
    def __init__(self, **kw):
        super().__init__(**kw)
        self.peaks = [0, 0.2, 0.8, 1]
        self.var = 0.001

    def __getitem__(self, i):
        # randomly choose one of the peaks
        ret = random.choice(self.peaks)
        ret = ret + random.gauss(0, self.var)
        return torch.tensor([ret])[:, None, None]

    def __len__(self):
        return 4000

    def __iter__(self):
        for i in range(len(self)):
            yield self[i]





In [3]:

m = OneDModel(64)
x = torch.randn((5, 1, 1, 1))
t = torch.rand((5,))
print(x, t)

# y = m(x, t)
# print(y)
timesteps = 512

diffusion = ProgressiveDistillationGaussianDiffusion(model=m, image_size=1, timesteps=timesteps)

ds = OneDDataset()
dl = torch.utils.data.DataLoader(ds, batch_size=256)

print(len(ds))
samples = [x[0, 0, 0].item() for x in ds]
print(samples[0:100])
# _ = plt.hist(samples, density=True, bins=100)
# plt.show()


tensor([[[[ 1.2041]]],


        [[[ 1.2833]]],


        [[[ 0.1578]]],


        [[[-0.7540]]],


        [[[ 1.3091]]]]) tensor([0.7824, 0.1892, 0.2214, 0.4067, 0.3026])
4000
[-5.5748303566360846e-05, 0.0011339361080899835, -0.0008620451553724706, 0.2001534104347229, 6.294403283391148e-05, 0.19829526543617249, -0.001527694403193891, 1.0003992319107056, -0.0016337418928742409, 0.00042412467882968485, 0.801693320274353, -0.0013211912009865046, 0.19989831745624542, 0.0011962804710492492, 0.7992130517959595, 0.0012381889391690493, 0.8004467487335205, 0.20031452178955078, 0.2004346251487732, 0.20086650550365448, 0.999267041683197, 0.7991926670074463, 0.00016145840345416218, -1.2448708730516955e-05, 0.7977547645568848, 1.0005247592926025, 0.799424946308136, 0.20096638798713684, 0.20065838098526, 0.00025729837943799794, 1.0013055801391602, 0.2005314826965332, 0.20017263293266296, 0.7994054555892944, 0.000996081274934113, 0.9979673027992249, 0.8002524971961975, 0.20043043792247772, 0.997195

In [4]:

step = 0
epochs = [4000]
_timesteps = timesteps
while _timesteps > 1:
    epochs.append(1000)
    _timesteps /= 2

_epochs = deepcopy(epochs)

device = torch.device("cuda:0")
diffusion.to(device)
done = False

optimizer = torch.optim.Adam(diffusion.model.parameters(), lr=1e-3, betas=(0.9, 0.99))

models = []


In [5]:

with tqdm(initial=step, total=sum(epochs) * len(dl)) as pbar:
    while not done:
        losses = []
        for batch in dl:
            batch = batch.to(device)

            loss = diffusion(batch)
            loss.backward()
            losses.append(loss.cpu().item())

            pbar.set_description(f'loss: {np.mean(losses):.4f}')

            optimizer.step()
            optimizer.zero_grad()

            step += 1
            pbar.update(1)

        epochs[0] -= 1
        if epochs[0] == 0:
            print(f"Saving model for {diffusion.jumpsize.item()} steps")
            models.append(deepcopy(diffusion))
            epochs.pop(0)
            if len(epochs) == 0:
                break
            print(f"doubling jump size: {diffusion.jumpsize.item()*2}")
            diffusion.double_jump_size()

        if step % (len(dl) * 50) == 0:
            print("")
                  
print("done training")


loss: 0.3550:   0%|▏                                                             | 822/208000 [00:06<26:38, 129.59it/s]

loss: 0.3424:   1%|▍                                                            | 1622/208000 [00:13<26:55, 127.77it/s]

loss: 0.3637:   1%|▋                                                            | 2423/208000 [00:19<26:48, 127.84it/s]

loss: 0.3549:   2%|▉                                                            | 3224/208000 [00:25<26:43, 127.68it/s]

loss: 0.3605:   2%|█▏                                                           | 4023/208000 [00:32<26:50, 126.67it/s]

...

loss: 0.0009:  99%|██████████████████████████████████████████████████████████▎| 205619/208000 [28:32<00:21, 110.88it/s]

loss: 0.0015:  99%|██████████████████████████████████████████████████████████▌| 206420/208000 [28:39<00:14, 111.75it/s]

loss: 0.0010: 100%|██████████████████████████████████████████████████████████▊| 207219/208000 [28:46<00:06, 112.59it/s]

loss: 0.0017: 100%|███████████████████████████████████████████████████████████| 208000/208000 [28:53<00:00, 119.98it/s]

Saving model for 512 steps
done training


In [ ]:
print(len(models))
hists = []
for model in models:
    sampled_images = model.sample(batch_size=2000)
    print(sampled_images.shape)  # (4, 3, 128, 128)
    sampled_images = sampled_images[:, 0, 0, 0]
    # print(sampled_images)
    hist, _ = np.histogram(sampled_images.cpu().numpy(), density=True, bins=200)
    hist /= hist.max()
    hists.append(hist)
    plt.hist(sampled_images.cpu().numpy(), density=True, bins=100)
    plt.show()
imdata = np.stack(hists)
plt.imshow(imdata)
plt.show()

In [ ]:
def show_trajectories(diffusion, faststeps=10, numtraj=5, range=(-1, 1)):
    x_T = np.linspace(range[0], range[1], numtraj)
    diffusion.is_ddim_sampling = True
    diffusion.ddim_sampling_eta = 0
    diffusion.sampling_timesteps = timesteps
    
    sampled_images, times, imgacc, x0acc = diffusion.ddim_sample((numtraj, 1,1,1), 
                                                                 x_T=torch.tensor(x_T)[:, None, None, None].to(torch.float)
                                                                     .to(diffusion.betas.device))
    
    print(len(times), len(imgacc))
    
    trajs = [[x_t_i] for x_t_i in unnormalize_to_zero_to_one(x_T)]
    
    for img in imgacc:
        img = img[:,0,0,0].cpu().numpy()
        for img_i, traj in zip(img, trajs):
            traj.append(img_i)
        
    fig = plt.figure(figsize = (15,10))
    ax = fig.add_subplot(111)
    for traj in trajs:
        ax.plot(times, traj, "blue")
    ax.set_title('trajectories')
    
    diffusion.sampling_timesteps = faststeps
    
    sampled_images, times, imgacc, x0acc = diffusion.ddim_sample((numtraj, 1,1,1), 
                                                                 x_T=torch.tensor(x_T)[:, None, None, None].to(torch.float)
                                                                     .to(diffusion.betas.device))
    
    print(len(times), len(imgacc))
    
    trajs = [[x_t_i] for x_t_i in unnormalize_to_zero_to_one(x_T)]
    
    for img in imgacc:
        img = img[:,0,0,0].cpu().numpy()
        for img_i, traj in zip(img, trajs):
            traj.append(img_i)
        
    for traj in trajs:
        ax.plot(times, traj, "ro-")
    ax.set_title('trajectories')

    plt.show()

    return trajs